In [1]:
!pip install -q --upgrade wandb

     |████████████████████████████████| 1.7 MB 2.7 MB/s 
     |████████████████████████████████| 170 kB 47.7 MB/s 
     |████████████████████████████████| 133 kB 57.8 MB/s 
     |████████████████████████████████| 97 kB 5.5 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 


In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("vincentwang25/G2Net")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.job_type)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
runs_df = runs_df.dropna()
CQT_test_part = runs_df[runs_df.name.str.contains("testCQT2")]

In [4]:
names = CQT_test_part["name"].str[len("e1-testCQT2-"):]
summary = CQT_test_part['summary'].apply(pd.Series)

In [5]:
summary

,_step,_runtime,[fold0] lr,_timestamp,[fold0] loss,[fold0] avg_train_loss,[fold0] epoch,[fold0] val_score,[fold0] avg_val_loss,_wandb
1,37.0,91.0,0.001448,1.630929e+09,0.487936,NaN,NaN,NaN,NaN,NaN
2,732.0,938.0,0.001023,1.630929e+09,0.398026,0.423147,2.0,0.865394,0.420312,NaN
3,1052.0,1055.0,0.000000,1.630928e+09,0.403164,0.404736,3.0,0.862851,0.424263,{'runtime': 1050}
4,1052.0,2302.0,0.000000,1.630928e+09,0.397309,0.405430,3.0,0.861391,0.425937,{'runtime': 2297}
5,35.0,34.0,0.001371,1.630925e+09,0.554929,NaN,NaN,NaN,NaN,NaN
6,1052.0,3417.0,0.000000,1.630898e+09,0.416189,0.410751,3.0,0.863747,0.425722,{'runtime': 3413}
7,1052.0,4025.0,0.000000,1.630898e+09,0.385773,0.413121,3.0,0.860147,0.430863,{'runtime': 4019}
8,1052.0,3302.0,0.000000,1.630895e+09,0.409507,0.415627,3.0,0.860208,0.432757,{'runtime': 3297}
9,1052.0,1322.0,0.000000,1.630891e+09,0.386652,0.401860,3.0,0.863733,0.421429,{'runtime': 1317}
10,1052.0,4026.0,0.000000,1.630894e+09,0.417161,0.420501,3.0,0.857304,0.436698,{'runtime': 4022}


In [6]:
summary['name'] = names

In [7]:
summary.dropna(inplace=True)
summary.drop_duplicates(subset='name',inplace=True)

In [8]:
summary = summary.sort_values('[fold0] val_score', ascending=False)

In [9]:
summary[~summary['name'].str.contains('500-')]

,_step,_runtime,[fold0] lr,_timestamp,[fold0] loss,[fold0] avg_train_loss,[fold0] epoch,[fold0] val_score,[fold0] avg_val_loss,_wandb,name
30,1052.0,2870.0,0.0,1.630874e+09,0.390242,0.405263,3.0,0.864901,0.422732,{'runtime': 2865},FM1024_HL32_BPO24_FS0.5
6,1052.0,3417.0,0.0,1.630898e+09,0.416189,0.410751,3.0,0.863747,0.425722,{'runtime': 3413},FM1024_HL32_BPO24_FS0.75
9,1052.0,1322.0,0.0,1.630891e+09,0.386652,0.401860,3.0,0.863733,0.421429,{'runtime': 1317},FM1024_HL32_BPO12_FS0.75
11,1052.0,1175.0,0.0,1.630890e+09,0.392985,0.403022,3.0,0.863367,0.423351,{'runtime': 1170},FM1024_HL40_BPO12_FS0.75
3,1052.0,1055.0,0.0,1.630928e+09,0.403164,0.404736,3.0,0.862851,0.424263,{'runtime': 1050},FM1024_HL48_BPO12_FS0.75
23,1052.0,1181.0,0.0,1.630879e+09,0.415630,0.410171,3.0,0.860772,0.429389,{'runtime': 1176},FM1024_HL64_BPO12_FS0.75
31,1052.0,1207.0,0.0,1.630872e+09,0.417903,0.413408,3.0,0.860416,0.429547,{'runtime': 1202},FM1024_HL64_BPO12_FS1
8,1052.0,3302.0,0.0,1.630895e+09,0.409507,0.415627,3.0,0.860208,0.432757,{'runtime': 3297},FM1024_HL32_BPO24_FS1
15,1052.0,944.0,0.0,1.630888e+09,0.398325,0.408140,3.0,0.860125,0.427240,{'runtime': 939},FM1024_HL56_BPO12_FS0.75
24,1052.0,888.0,0.0,1.630878e+09,0.414132,0.415828,3.0,0.858983,0.432488,{'runtime': 884},FM500_HL64_BPO12_FS1


In [10]:
summary[summary['name'].str.contains('500-')]

,_step,_runtime,[fold0] lr,_timestamp,[fold0] loss,[fold0] avg_train_loss,[fold0] epoch,[fold0] val_score,[fold0] avg_val_loss,_wandb,name
27,1052.0,2683.0,0.0,1.630877e+09,0.407226,0.407836,3.0,0.865371,0.422951,{'runtime': 2678},500-FM500_HL32_BPO24_FS0.5
12,1052.0,1322.0,0.0,1.630890e+09,0.396493,0.403796,3.0,0.863423,0.423175,{'runtime': 1316},500-FM500_HL32_BPO12_FS0.75
4,1052.0,2302.0,0.0,1.630928e+09,0.397309,0.405430,3.0,0.861391,0.425937,{'runtime': 2297},500-FM500_HL40_BPO12_FS0.75
7,1052.0,4025.0,0.0,1.630898e+09,0.385773,0.413121,3.0,0.860147,0.430863,{'runtime': 4019},500-FM500_HL32_BPO24_FS0.75
16,1052.0,1059.0,0.0,1.630887e+09,0.410509,0.407290,3.0,0.859968,0.428278,{'runtime': 1054},500-FM500_HL48_BPO12_FS0.75
19,1052.0,968.0,0.0,1.630881e+09,0.426866,0.413063,3.0,0.859774,0.431271,{'runtime': 963},500-FM500_HL64_BPO12_FS0.75
29,1052.0,984.0,0.0,1.630875e+09,0.420339,0.416253,3.0,0.859356,0.431251,{'runtime': 979},500-FM500_HL64_BPO12_FS1
17,1052.0,1035.0,0.0,1.630884e+09,0.407528,0.410615,3.0,0.858962,0.428839,{'runtime': 1030},500-FM500_HL56_BPO12_FS0.75
10,1052.0,4026.0,0.0,1.630894e+09,0.417161,0.420501,3.0,0.857304,0.436698,{'runtime': 4022},500-FM500_HL32_BPO24_FS1
22,1052.0,690.0,0.0,1.630879e+09,0.418464,0.415734,3.0,0.856536,0.434739,{'runtime': 685},500-FM500_HL64_BPO12_FS0.5


1. filter scale 0.75 works the best
2. larger picture works better (but maybe rescale can achieve the same effect)
3. lowpass 500 helps, but in a small marge (<=5 bps)